# Sensitivity analysis

In [1]:
from SALib.sample import saltelli, fast_sampler
from SALib.sample import ff as ffs
from SALib.analyze import sobol, fast, ff
from SALib.plotting.bar import plot as barplot
import numpy as np
import pandas as pd
import shutil
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

In [44]:
# parameter dictionary
problem = {
    'num_vars': 3,
    'names': ['random-mating', 'leakage', 'fertility'],
    'bounds': [[0, 0.75],
               [0, 0.50],
               [0, 0.30]]
}

# Sobol

In [ ]:
sample_parameters = False
if (sample_parameters):
    param_values = saltelli.sample(problem, 2**12)
    t = pd.DataFrame(param_values)
    t.columns = ["random-mating", "leakage", "fertility"]
    t.to_csv("../models/fertility-BMI/data/param-sobol.csv", index=False)
else:
    t = pd.read_csv("../models/fertility-BMI/data/param-sobol.csv")
len(t)

In [ ]:
df = pd.read_csv("../output/data/results-sobol.csv", sep = ";")
params = pd.read_csv("../output/data/param-sobol.csv", sep = ";")
df = df.merge(params, on = ["iteration", "replicate"])
df["iteration"] = pd.to_numeric(df["iteration"])
df = df.sort_values(by=['iteration'])
df = df.loc[df["iteration"] <= len(t),:]
Y = df["g4"].to_numpy()
len(Y)

In [ ]:
Si = sobol.analyze(problem, Y, print_to_console=True)

In [ ]:
total, first, second = Si.to_df()

In [ ]:
st = barplot(total)
plt.savefig("../output/plots/sobol-st.pdf", bbox_inches='tight')
s1 = barplot(first)
plt.savefig("../output/plots/sobol-s1.pdf", bbox_inches='tight')
s2 = barplot(second)
plt.savefig("../output/plots/sobol-s2.pdf", bbox_inches='tight')

shutil.copy("../output/plots/sobol-st.pdf", "../manuscript/plots/")
shutil.copy("../output/plots/sobol-s1.pdf", "../manuscript/plots/")
shutil.copy("../output/plots/sobol-s2.pdf", "../manuscript/plots/")

In [ ]:
fig = px.scatter_3d(df, x='random_mating', y='fertility_factor', z='leakage',
              color='g4', color_continuous_scale = px.colors.sequential.OrRd)

fig.update_traces(marker=dict(size=4, opacity = 0.4),
                  selector=dict(mode='markers'))
fig.update_layout(scene = dict(
                    xaxis_title='random mating',
                    yaxis_title='fertility',
                    zaxis_title='leakage'),
                    width=700,
                    margin=dict(r=10, b=10, l=10, t=10), 
                    coloraxis_colorbar=dict(title="Obese")
                    )
fig.show()

In [ ]:
# fig.write_image("../output/plots/3D-scatter-sobol.pdf")
shutil.copy("../output/plots/3D-scatter-sobol.png", "../manuscript/plots/")

# EFast

In [ ]:

sample_parameters = False

if (sample_parameters):
    param_values = fast_sampler.sample(problem, 2000)
    t = pd.DataFrame(param_values)
    t.columns = ["random-mating", "leakage", "fertility"]
    t.to_csv("../models/fertility-BMI/data/param-efast.csv", index=False)
else:
    t = pd.read_csv("../models/fertility-BMI/data/param-efast.csv")
t.shape

In [ ]:
df = pd.read_csv("../output/data/results-efast.csv", sep = ";")
params = pd.read_csv("../output/data/param-efast.csv", sep = ";")
df = df.merge(params, on = ["iteration", "replicate"])
df["iteration"] = pd.to_numeric(df["iteration"])
df = df.sort_values(by=['iteration'])
df = df.loc[df["iteration"] <= len(t),:]
Y = df["g4"].to_numpy()
len(Y)

In [ ]:
df.g4.describe()

In [ ]:
Sif = fast.analyze(problem, Y, print_to_console=True)

In [ ]:
Sif.plot()
plt.savefig("../output/plots/efast.pdf", bbox_inches='tight')

In [ ]:
shutil.copy("../output/plots/efast.pdf", "../manuscript/plots/")

In [ ]:
fig = px.scatter_3d(df, x='random_mating', y='fertility_factor', z='leakage',
              color='g4', color_continuous_scale = px.colors.sequential.OrRd)

fig.update_traces(marker=dict(size=4, opacity = 0.7),
                  selector=dict(mode='markers'))
fig.update_layout(scene = dict(
                    xaxis_title='random mating',
                    yaxis_title='fertility',
                    zaxis_title='leakage'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10), 
                    coloraxis_colorbar=dict(title="Obese")
                    )
fig.show()

In [ ]:
#fig.write_image("../output/plots/3D-scatter-efast.pdf")
shutil.copy("../output/plots/3D-scatter-efast.png", "../manuscript/plots/")

# eFast replicates

In [54]:

sample_parameters = True

if (sample_parameters):
    param_values = fast_sampler.sample(problem, 500)
    
    t = pd.DataFrame(param_values)
    t.columns = ["random-mating", "leakage", "fertility"]
    t.to_csv("../models/fertility-BMI/data/param-rep-efast.csv", index=False)
else:
    t = pd.read_csv("../models/fertility-BMI/data/param-rep-efast.csv")
t.shape

(1500, 3)

In [45]:
df = pd.read_csv("../output/data/results-rep-efast.csv", sep = ";")
params = pd.read_csv("../output/data/param-rep-efast.csv", sep = ";")
df = df.merge(params, on = ["iteration", "replicate"])
df["iteration"] = pd.to_numeric(df["iteration"])
df["replicate"] = pd.to_numeric(df["replicate"])
df = df.sort_values(by=['iteration'])
df = df.loc[df["iteration"] <= len(t),:]

In [46]:
replicates = df.replicate.sort_values().unique()
Y = {}
for i in replicates:
    tY = df.loc[df["replicate"] == i].g4.to_numpy()
    Y[i] = fast.analyze(problem, tY, print_to_console=False)

In [58]:
a = df.loc[df["replicate"] == 3].g4.to_numpy()
tsif = fast.analyze(problem, a, print_to_console=True)
tsif

                     S1        ST   S1_conf   ST_conf
random-mating  0.002703  0.188205  0.013305  0.032870
leakage        0.171588  0.564932  0.014168  0.040110
fertility      0.020447  0.842152  0.015333  0.038622


{'S1': [0.002703358634055152, 0.1715883182255363, 0.02044736212530417],
 'ST': [0.1882053640551078, 0.5649321292003946, 0.8421521666383727],
 'S1_conf': [0.013304969885468676, 0.01416756647697069, 0.015333401502273877],
 'ST_conf': [0.03287032755328449, 0.04010963959753077, 0.038622387448406674],
 'names': ['random-mating', 'leakage', 'fertility']}

In [59]:
Y[1]["names"]

['random-mating', 'leakage', 'fertility']

In [60]:

columns = Y[1]["names"]
S1 = []
ST = []

for value in Y.values():
    S1.append(value["S1"])
    ST.append(value["ST"])

In [61]:
S1 = pd.DataFrame(np.row_stack(S1))
S1.columns = columns
ST = pd.DataFrame(np.row_stack(ST))
ST.columns = columns

In [62]:
S1.describe()

,random-mating,leakage,fertility
count,30.000000,30.000000,30.000000
mean,0.002579,0.152408,0.016757
std,0.001447,0.019487,0.009988
min,0.000613,0.122621,0.003499
25%,0.001590,0.140918,0.008607
50%,0.002538,0.151518,0.014552
75%,0.002862,0.161634,0.023527
max,0.007554,0.221078,0.035767


In [63]:
ST.describe()

,random-mating,leakage,fertility
count,30.000000,30.000000,30.000000
mean,0.202531,0.530228,0.869335
std,0.012184,0.019337,0.018003
min,0.182863,0.496486,0.826141
25%,0.192890,0.518205,0.859247
50%,0.201479,0.526604,0.869667
75%,0.212559,0.540596,0.881739
max,0.231200,0.573341,0.911103
